___

#### City Recommender System
___
<h3 align="Left">by Mohammd Aboutorabi,Fatemeh Rahideh,Mohammad Yousofvand
 </h3> 

___


In [1]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

%matplotlib inline

#### Import Dataframe

In [2]:
norm_df = pd.read_csv('https://s3.us-east-2.amazonaws.com/www.findingmyschittscreek.com/Data/normalized_df_sub.csv', index_col=0);

norm_df

City  Population  Median Property Value  Median Age  \
0     Los Angeles, CA    0.107459               0.025957    0.044300   
1      Washington, DC    0.072403               0.019314    0.044300   
2    Philadelphia, PA    0.071481               0.012511    0.049097   
3         Seattle, WA    0.056660               0.021105    0.044018   
4     Minneapolis, MN    0.054225               0.013131    0.044864   
..                ...         ...                    ...         ...   
298      Victoria, TX    0.005608               0.005928    0.042325   
299          Rome, GA    0.005483               0.005765    0.047686   
300          Ames, IA    0.005455               0.008454    0.012697   
301      Columbus, IN    0.004690               0.006552    0.047122   
302        Casper, WY    0.004690               0.009791    0.042043   

     Commute Time  Health Care    Safety  Pollution  Summer Temp  Fall Temp  \
0        0.086313     0.013948  0.006723  -0.020365    -0.004250  -0.030698   
1        0.102006     0.016999  0.004647  -0.012046    -0.006787  -0.024103   
2        0.083543     0.016431  0.004030  -0.015487    -0.007104  -0.024463   
3        0.086774     0.017643  0.006505  -0.008580    -0.002664  -0.021105   
4        0.061388     0.019231  0.006039  -0.008187    -0.005265  -0.017388   
..            ...          ...       ...        ...          ...        ...   
298      0.041541     0.016221  0.009219  -0.004715    -0.009958  -0.034176   
299      0.048464     0.012706  0.005675  -0.020463    -0.008182  -0.027581   
300      0.028155     0.014372  0.013005  -0.012792    -0.005772  -0.019906   
301      0.038310     0.017840  0.007536  -0.007733    -0.006406  -0.022544   
302      0.027232     0.017770  0.011892  -0.004322    -0.003742  -0.015349   

     ...  Whisky Bar  Windmill  Wine Bar  Wine Shop  Winery  Wings Joint  \
0    ...         0.0       0.0  0.000000       -0.0    -0.0     0.000000   
1    ...         0.0       0.0  0.027766       -0.0    -0.0     0.000000   
2    ...         0.0       0.0  0.000000       -0.0    -0.0     0.004243   
3    ...         0.0       0.0  0.009255       -0.0    -0.0     0.000000   
4    ...         0.0       0.0  0.000000       -0.0    -0.0     0.000000   
..   ...         ...       ...       ...        ...     ...          ...   
298  ...         0.0       0.0  0.000000       -0.0    -0.0     0.004243   
299  ...         0.0       0.0  0.000000       -0.0    -0.0     0.004243   
300  ...         0.0       0.0  0.009255       -0.0    -0.0     0.008487   
301  ...         0.0       0.0  0.000000       -0.0    -0.0     0.004243   
302  ...         0.0       0.0  0.009255       -0.0    -0.0     0.000000   

     Women's Store  Yoga Studio       Zoo   density  
0             -0.0     0.000000  0.000000 -0.015852  
1             -0.0     0.016830  0.000000 -0.021296  
2             -0.0     0.000000  0.000000 -0.022123  
3             -0.0     0.033660  0.000000 -0.016058  
4             -0.0     0.008415  0.000000 -0.014470  
..             ...          ...       ...       ...  
298           -0.0     0.000000  0.010934 -0.002835  
299           -0.0     0.000000  0.000000 -0.001578  
300           -0.0     0.000000  0.000000 -0.004032  
301           -0.0     0.000000  0.000000 -0.002599  
302           -0.0     0.000000  0.000000 -0.003561  

[303 rows x 635 columns]

#### Variables Considered

This recommendation system takes into consideration a multitude of variables scraped, selected, processed and modeled in the following notebooks:
1. <a href="https://github.com/eliasmelul/finding_schitts/blob/master/weather_data_FMSC.ipynb">Weather Data</a> --- <a href="https://s3.us-east-2.amazonaws.com/www.findingmyschittscreek.com/Data/final_weather_data.csv">Data Direct Download</a>
2. <a href="https://github.com/eliasmelul/finding_schitts/blob/master/general_data_FMSC.ipynb">General Socioeconomic Data</a> --- <a href="https://s3.us-east-2.amazonaws.com/www.findingmyschittscreek.com/Data/scraped_datausa.csv">Data Direct Download</a>
3. <a href="https://github.com/eliasmelul/finding_schitts/blob/master/Foursquare_data_FMSC.ipynb">Venue Data: Foursquare</a> --- <a href="https://s3.us-east-2.amazonaws.com/www.findingmyschittscreek.com/Data/one_hot_encoded_usa_FS.csv">Data Direct Download</a>
4. <a href="https://github.com/eliasmelul/finding_schitts/blob/master/EDA_FMSC.ipynb">Exploratory Data Analysis and Preprocessing</a> --- <a href="https://s3.us-east-2.amazonaws.com/www.findingmyschittscreek.com/Data/allDataCombined.csv">Data Direct Download</a>
5. <a href="https://github.com/eliasmelul/finding_schitts/blob/master/modeling_LM_FMSC.ipynb">Modeling: Weights</a> --- <a href="https://s3.us-east-2.amazonaws.com/www.findingmyschittscreek.com/Data/normalized_df_sub.csv">Data Direct Download</a>

---

# Recommendation System

### Function Definition


In [3]:
def from_city_cosSim(data, name):

    Xs = data[data.City == name].drop('City',axis=1)
    Col_A = data[data.City != name].City
    Ys = data[data.City != name].drop('City',axis=1)

    cosSim = cosine_similarity(X=Xs,Y=Ys)
    sim = list(cosSim[0])
    cty = list(Col_A)
    comb = {"City":cty,"Similarity":sim}
    dfdf = pd.DataFrame(comb).reset_index()

    # Adding a row with the Y City
    currCity = {"City":name,"Similarity":1}
    curr = pd.DataFrame(currCity, index=[0])

    # Concatenate to finalize DF
    dfdf = pd.concat([dfdf,curr], sort=False).reset_index(drop=True).drop('index',axis=1)
    return(dfdf)

In [4]:
def input_cities(numShow=10):
    # create class that defines cities
    class rated_city:
        def __init__(self, city):
            self.city = city
    
    #Loop to input cities based on the user
    add_city = True
    userInput = []
    w = 0
    while add_city == True:
        city_name = input("City (Include state - Ex. New York, NY): ")
        userInput.append(city_name)
        simSim = from_city_cosSim(data=norm_df, name=city_name)
        try:
            cosSim = cosSim.merge(simSim, how='inner', on='City')
        except:
            cosSim = simSim

        print(simSim)
        
        city = rated_city(city_name)
        cont = input("Do you want to include another city?")
        add_city = cont.lower() in ['yes','true','of course','y','si','1']
        w+=1
    
    simCols = cosSim.drop("City", axis=1)
    cits = cosSim.City
        
    for i, row in simCols.iterrows():
        simCols.at[i,'SumVal'] = row.sum()/w
    simi = simCols.SumVal
    
    out = {"City":cits,"Score":simi}
    out = pd.DataFrame(out).set_index("City").drop(userInput)
    out = out.sort_values('Score', ascending=False)
    
    return out.head(numShow)      

## Recommendations

---
**Use:** to use this system, all you have to do is run the _input_cities()_ function. This will return a list of the most similar cities - the recommendations!

In [7]:
input_cities ()

                 City  Similarity
0     Los Angeles, CA    0.717961
1      Washington, DC    0.760189
2    Philadelphia, PA    0.747166
3         Seattle, WA    0.760618
4     Minneapolis, MN    0.743887
..                ...         ...
298          Rome, GA    0.382224
299          Ames, IA    0.363977
300      Columbus, IN    0.578359
301        Casper, WY    0.356643
302      New York, NY    1.000000

[303 rows x 2 columns]


Score
City                      
Chicago, IL       0.799584
Seattle, WA       0.760618
Washington, DC    0.760189
Philadelphia, PA  0.747166
Pittsburgh, PA    0.746654
Minneapolis, MN   0.743887
Modesto, CA       0.726210
Baltimore, MD     0.724874
Tampa, FL         0.720664
Los Angeles, CA   0.717961